In [ ]:
import sys
import os
import h5py
import json
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

from stimuli_f0_labels import get_f0_bins, f0_to_label

fn = '/om4/group/mcdermott/user/msaddler/pitchnet_dataset/pitchnetDataset/assets/data/processed/dataset_2019-11-22-2300/PND_sr32000_v08.hdf5'
# fn = '/om4/group/mcdermott/user/msaddler/pitchnet_dataset/pitchnetDataset/assets/data/processed/dataset_2019-11-16-2300/PND_sr32000_v07.hdf5'
# fn = '/om4/group/mcdermott/user/msaddler/pitchnet_dataset/pitchnetDataset/assets/data/processed/dataset_2019-08-16-1200/PND_sr32000_v04.hdf5'

f = h5py.File(fn, 'r')
for v in f.values():
    print(v)
for v in f['config'].values():
    print(v)

file_indexes = f['source_file_index'][:]
segment_indexes = f['source_file_row'][:]
f0_values = f['nopad_f0_mean'][:]
source_file_encoding_dict = f['config/source_file_encoding_dict'][0]
source_file_encoding_dict = source_file_encoding_dict.replace('"', '"""')
source_file_encoding_dict = source_file_encoding_dict.replace('\'', '"')
source_file_encoding_dict = json.loads(source_file_encoding_dict)

f.close()


In [ ]:
file_index_to_filename_map = {}
for key in source_file_encoding_dict.keys():
    file_index_to_filename_map[source_file_encoding_dict[key]] = os.path.basename(key)


f0_bins = get_f0_bins()
dataset_separated_histograms = {}
dataset_separated_unique_segments = {}
dataset_separated_total_segments = {}

for file_index in np.unique(file_indexes):
    f0_values_from_file_idx = f0_values[file_indexes == file_index]
    segment_indexes_from_file_idx = segment_indexes[file_indexes == file_index]
    counts, bins = np.histogram(f0_values_from_file_idx, bins=f0_bins)
    dset_key = file_index_to_filename_map[file_index]
    dataset_separated_histograms[dset_key] = counts
    dataset_separated_unique_segments[dset_key] = np.unique(segment_indexes_from_file_idx).shape[0]
    dataset_separated_total_segments[dset_key] = segment_indexes_from_file_idx.shape[0]

dataset_details = {
    'RWC': {
        'key': 'f0_segments_2019AUG16_rwc.hdf5',
        'plot_kwargs': {'color': [0, 0.8, 0]},
    },
    'NSYNTH': {
        'key': 'f0_segments_2019AUG16_nsynth.hdf5',
        'plot_kwargs': {'color': [0, 0.6, 0]}
    },
    'WSJ': {
        'key': 'f0_segments_2019AUG16_wsj.hdf5',
        'plot_kwargs': {'color': [0.6, 0.6, 0.6]}
    },
    'SWC': {
        'key': 'f0_segments_2019AUG16_swc.hdf5',
        'plot_kwargs': {'color': [0.4, 0.4, 0.4]}
    },
    'CSLUKIDS': {
        'key': 'f0_segments_2019NOV16_cslu_kids.hdf5',
        'plot_kwargs': {'color': [0.2, 0.2, 0.2]}
    },
    'CMUKIDS': {
        'key': 'f0_segments_2019NOV22_cmu_kids.hdf5',
        'plot_kwargs': {'color': [0.8, 0.8, 0.8]}
    },
}

dataset_list = ['RWC', 'NSYNTH', 'WSJ', 'SWC', 'CSLUKIDS', 'CMUKIDS']
dataset_list.reverse()

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 6))
x = np.arange(0, len(counts))
bottom = np.zeros_like(x)
for dataset in dataset_list:
    key = dataset_details[dataset]['key']
    if key in dataset_separated_histograms.keys():
        y = dataset_separated_histograms[key]
        plot_kwargs = dataset_details[dataset]['plot_kwargs']
        label = '{:s} ({} total; {} unique; {:.1f} mean repeats)'.format(
            dataset,
            dataset_separated_total_segments[key],
            dataset_separated_unique_segments[key],
            dataset_separated_total_segments[key] / dataset_separated_unique_segments[key])
        ax.fill_between(x, y1=bottom, y2=bottom+y, **plot_kwargs, lw=0, label=label)
        bottom = bottom + y
    else:
        print(key)

ax.legend(loc='upper right', framealpha=1, facecolor='w', edgecolor='w', fontsize=10)
ax.set_xlim([x[0], x[-1]])
ax.set_ylim([0, np.max(bottom)])
ax.set_ylabel('Number of stimuli')
ax.set_xlabel('F0 bin (Hz)')
class_indexes = np.linspace(x[0], x[-1], 15, dtype=int)
f0_class_labels = ['{:.0f}'.format(_) for _ in f0_bins[class_indexes]]
ax.set_xticks(class_indexes)
ax.set_xticklabels(f0_class_labels)
plt.tight_layout()
plt.show()

# save_dir = '/om2/user/msaddler/pitchnet/assets_psychophysics/figures/archive_2019_12_05_PNDv08_archSearch01/'
# save_fn = '2019NOV27_PND_v08_dataset_composition.pdf'
# print(os.path.join(save_dir, save_fn))
# fig.savefig(os.path.join(save_dir, save_fn), bbox_inches='tight')


In [ ]:
# Check how many bins are spanned by speech-only and music-only datasets
f0_bin_min=80
f0_bin_max=1e3
f0_min=80
f0_max=450.91752190019395
binwidth_in_octaves=1/192
f0_values = np.arange(f0_min, f0_max+1e-2, 1e-2)
f0_bins = get_f0_bins(f0_min=f0_bin_min, f0_max=f0_bin_max, binwidth_in_octaves=binwidth_in_octaves)
f0_labels = f0_to_label(f0_values, f0_bins, right=False)
# Slightly hacky way to determine the correct value of f0_max to ensure all bins are equally wide
f0_min_label = np.squeeze(np.argwhere(f0_bins >= f0_min))[0]
f0_max_label = np.squeeze(np.argwhere(f0_bins < f0_max))[-1] + 1
f0_min_label, f0_max_label


In [ ]:
import sys
import os
import h5py
import glob
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/packages/msutil')
import util_stimuli
import util_misc
import util_figures


regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/PND_sr32000_v08_*.hdf5'

list_fn = sorted(glob.glob(regex_fn))
fn = list_fn[-1]

with h5py.File(fn, 'r') as f:
    sr = f['sr'][0]
    IDX = np.random.randint(0, f['nopad_f0_mean'].shape[0])
    f0 = f['nopad_f0_mean'][IDX]
    y_fg = util_stimuli.set_dBSPL(f['stimuli/signal'][IDX], 60.0)
    y_bg = util_stimuli.set_dBSPL(f['stimuli/noise'][IDX], 60.0)


fxx, pxx = util_stimuli.power_spectrum(y_fg, sr)
fenv, penv = util_stimuli.get_spectral_envelope_lp(y_fg, sr, M=6)
penv = penv - penv.max() + pxx.max()

print(util_stimuli.get_dBSPL(y_fg))
print(util_stimuli.get_dBSPL(y_bg))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 2.5))
ax.plot(fxx, pxx, color='k', lw=1.0)
ax.plot(fenv, penv, color='r', lw=1.0)
ax = util_figures.format_axes(ax,
                              xscale='linear',
                              str_xlabel='Frequency (Hz)',
                              str_ylabel='Power (dB SPL)',
                              xlimits=[40, sr/2],
                              ylimits=None,
                              spines_to_hide=['right', 'top'])
plt.show()

ipd.display(ipd.Audio(y_fg, rate=sr))


In [ ]:
y = y_fg

t = np.arange(0, len(y)) / sr
x = np.zeros_like(y)
for f in np.arange(f0, sr/2, f0):
    x = x + np.sin(2*np.pi*f*t)
# x = np.random.randn(*t.shape)

b_lp, a_lp = util_stimuli.get_spectral_envelope_lp_coefficients(y, M=6)
x = scipy.signal.lfilter(b_lp, a_lp, x)


x = util_stimuli.set_dBSPL(x, 60.0)

fyy, pyy = util_stimuli.power_spectrum(y, sr)
fxx, pxx = util_stimuli.power_spectrum(x, sr)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 2.5))
ax.plot(fyy, pyy, color='k', lw=1.0)
ax.plot(fxx, pxx, color='r', lw=1.0)

ax = util_figures.format_axes(ax,
                              xscale='linear',
                              str_xlabel='Frequency (Hz)',
                              str_ylabel='Power (dB SPL)',
                              xlimits=[40, sr/2],
                              ylimits=None,
                              spines_to_hide=['right', 'top'])
plt.show()

plt.figure(figsize=(8, 1.5))
plt.plot(t, y, color='k')
plt.plot(t, x, color='r')
plt.show()

ipd.display(ipd.Audio(y, rate=sr))
ipd.display(ipd.Audio(x, rate=sr))


In [ ]:
import sys
import os
import h5py
import json
import glob
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd
sys.path.append('/packages/msutil')
import util_stimuli
import util_misc
import util_figures


# regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10_filter_signalLPv01/SPECTRAL_STATISTICS_v00/*.hdf5'
# regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/SPECTRAL_STATISTICS_v00/*.hdf5'
# regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_mfcc/matchedPNDv08_snr_neg10pos10_phase0/SPECTRAL_STATISTICS_v00/*.hdf5'
regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_mfcc/PNDv08negated12_TLASmatched12_snr_neg10pos10_phase3/SPECTRAL_STATISTICS_v00/*.hdf5'
list_fn = sorted(glob.glob(regex_fn))

list_key = ['stimuli/signal', 'stimuli/noise']

dict_mfcc = {key: [] for key in list_key}
dict_mean_spectra = {}

for itr_fn, fn in enumerate(list_fn):
    with h5py.File(fn, 'r') as f:
        sr = f['sr'][0]
        freqs = f['freqs'][0]
        nopad_start = f['nopad_start'][0]
        nopad_end = f['nopad_end'][0]
        for key in list_key:
            if itr_fn == 0:
                dict_mean_spectra[key] = {
                    'freqs': freqs,
                    'summed_power_spectrum': np.zeros_like(freqs),
                    'count': 0,
                    'nfft': nopad_end - nopad_start,
                }
            nrows = f[key + '_power_spectrum'].shape[0]
            nrows_steps = np.linspace(0, nrows, 2, dtype=int)
            for nrow_start, nrow_end in zip(nrows_steps[:-1], nrows_steps[1:]):
                all_spectra = f[key + '_power_spectrum'][nrow_start:nrow_end]
#                 TRUNCATE = -20
#                 all_spectra[all_spectra < TRUNCATE] = TRUNCATE
                IDX = np.isfinite(np.sum(all_spectra, axis=1))
                dict_mean_spectra[key]['summed_power_spectrum'] += np.sum(all_spectra[IDX], axis=0)
                dict_mean_spectra[key]['count'] += np.sum(IDX, axis=0)
            
            dict_mfcc[key].append(f[key + '_mfcc'][:])
        if itr_fn % 5 == 0:
            print(itr_fn, os.path.basename(fn), dict_mean_spectra[key]['count'])

for key in list_key:
    print('concatenating {} mfcc arrays'.format(key))
    dict_mfcc[key] = np.concatenate(dict_mfcc[key], axis=0)


In [ ]:
results_dict = {}
for key in sorted(dict_mfcc.keys()):
    mfcc_cov = np.cov(dict_mfcc[key], rowvar=False)
    mfcc_mean = np.mean(dict_mfcc[key], axis=0)
    results_dict[key] = {
        'mfcc_mean': mfcc_mean,
        'mfcc_cov': mfcc_cov,
        'sr': sr,
        'mean_power_spectrum': dict_mean_spectra[key]['summed_power_spectrum'] / dict_mean_spectra[key]['count'],
        'mean_power_spectrum_freqs': dict_mean_spectra[key]['freqs'],
        'mean_power_spectrum_count': dict_mean_spectra[key]['count'],
        'mean_power_spectrum_n_fft': dict_mean_spectra[key]['nfft'],
    }
    
#     results_dict[key]['mean_power_spectrum'] = 10*np.log10(results_dict[key]['mean_power_spectrum'])
    print(results_dict[key]['mean_power_spectrum'].max())
    print(results_dict[key]['mean_power_spectrum'].min())
fn_results_dict = os.path.join(os.path.dirname(fn), 'results_dict.json')
with open(fn_results_dict, 'w') as f:
    json.dump(results_dict, f, sort_keys=True, cls=util_misc.NumpyEncoder)
print(fn_results_dict)

for k0 in sorted(results_dict.keys()):
    for k1 in sorted(results_dict[k0].keys()):
        val = np.array(results_dict[k0][k1])
        if len(val.reshape([-1])) > 10:
            print(k0, k1, val.shape)
        else:
            print(k0, k1, val)

# nvars = dict_mfcc[key].shape[1]
# ncols = 4
# nrows = int(np.ceil(nvars/ncols))
# fig, ax = plt.subplots(ncols=ncols,
#                        nrows=nrows,
#                        figsize=(3*ncols, 2*nrows))
# ax = ax.reshape([-1])

# for ax_idx in range(nvars):
#     bins = 100#np.linspace(-2.5, 2.5, 100)
#     for key in sorted(results_dict.keys()):
#         vals = dict_mfcc[key][:, ax_idx]
#         ax[ax_idx].hist(vals, bins=bins, alpha=0.5)
#     ax[ax_idx] = util_figures.format_axes(ax[ax_idx],
#                                           str_xlabel='mfcc {}'.format(ax_idx + 1),
#                                           str_ylabel='Count',
#                                           xlimits=None,
#                                           ylimits=None)
    
# for ax_idx in range(nvars, ax.shape[0]):
#     ax[ax_idx].axis('off')

# plt.tight_layout()
# plt.show()


In [ ]:
import sys
import os
import h5py
import json
import glob
import numpy as np
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/packages/msutil')
import util_stimuli
import util_misc
import util_figures

data_dir = '/om/scratch/Fri/msaddler/data_pitchnet/'
basename = 'SPECTRAL_STATISTICS_v00/results_dict_v00.json'
list_dataset_tag = [
    ('PND_v08/noise_TLAS_snr_neg10pos10', 'Natural sounds'),
#     'PND_v08/noise_TLAS_snr_neg10pos10_filter_signalLPv01', 'Natural sounds (lowpass)'),
#     'PND_v08/noise_TLAS_snr_neg10pos10_filter_signalHPv00', 'Natural sounds (highpass)'),
#     'PND_v08inst/noise_TLAS_snr_neg10pos10',
#     'PND_v08spch/noise_TLAS_snr_neg10pos10',
    ('PND_mfcc/PNDv08matched12_TLASmatched12_snr_neg10pos10_phase0', 'Synthetic (12-MFCC-matched to natural)'),
#     'PND_mfcc/negatedPNDv08_snr_neg10pos10_phase0',
#     ('PND_mfcc/debug', 'Synthetic (flat spectrum)'),
#     ('PND_mfcc/PNDv08matched12_TLASmatched12_snr_neg10pos10_phase3', 'Synthetic (12-MFCC-matched)'),
#     ('PND_mfcc/PNDv08negated12_TLASmatched12_snr_neg10pos10_phase3', 'Synthetic (12-MFCC-negated)'),
]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 7.5))

clist = 'krbmgyc'
for cidx, (dataset_tag, label_tag) in enumerate(list_dataset_tag):
    fn_results_dict = os.path.join(data_dir, dataset_tag, basename)
    with open(fn_results_dict, 'r') as f:
        results_dict = json.load(f)
    
    for key in sorted(results_dict.keys()):
        MEAN_FXX = np.array(results_dict[key]['mean_power_spectrum_freqs'])
        MEAN_PXX = np.array(results_dict[key]['mean_power_spectrum_envelope'])
#         MEAN_PXX -= MEAN_PXX.max()
        
#         sr = np.array(results_dict[key]['sr'])
#         nfft = results_dict[key]['mean_power_spectrum_n_fft']
#         mfcc_mean = np.array(results_dict[key]['mfcc_mean'])
#         mfcc_mean[12:] = 0
#         M = librosa.filters.mel(sr, nfft, n_mels=len(mfcc_mean))
#         Minv = np.linalg.pinv(M)
#         power_spectrum = util_stimuli.get_power_spectrum_from_mfcc(mfcc_mean, Minv) 
#         MEAN_FXX = np.fft.rfftfreq(nfft, d=1/sr)
#         MEAN_PXX = 10*np.log10(power_spectrum)
        
        color = clist[cidx]
        ls = '-'
        if 'noise' in key:
            ls = '--'
        ax.plot(MEAN_FXX,
                MEAN_PXX,
                label='{} : {}'.format(label_tag, key),
                lw=2.5,
                color=color,
                ls=ls)
        
        MEAN_MFCC = np.array(results_dict[key]['mean_power_spectrum_freqs'])

    ax.legend(loc='lower left')
    ax = util_figures.format_axes(ax,
                                  xscale='log',
                                  str_xlabel='Frequency (Hz)',
                                  str_ylabel='Power (dB)',
                                  xlimits=[40, None],
                                  ylimits=[-40, None],
                                  spines_to_hide=['right', 'top'])
plt.show()


In [ ]:
import sys
import os
import h5py
import json
import glob
import copy
import pdb
import numpy as np
import scipy.signal
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/packages/msutil')
import util_stimuli
import util_misc
import util_figures

fn_results_dict = '/om/scratch/Mon/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/SPECTRAL_STATISTICS_v00/results_dict.json'
with open(fn_results_dict, 'r') as f:
    results_dict = json.load(f)

N = 1
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7.5, 5.0))
for itrN in range(N):
    for key in sorted(results_dict.keys())[1:]:
        mfcc_mean = np.array(results_dict[key]['mfcc_mean'])
        mfcc_cov = np.array(results_dict[key]['mfcc_cov'])
        sr = results_dict[key]['sr']
        dur = 0.150

        nfft = int(dur*sr)
        M = librosa.filters.mel(sr, nfft, n_mels=len(mfcc_mean))
        Minv = np.linalg.pinv(M)
        
        mfcc = np.random.multivariate_normal(mfcc_mean, mfcc_cov)
        mfcc[0:] = 0
        power_spectrum = util_stimuli.get_power_spectrum_from_mfcc(mfcc, Minv)
        power_spectrum_freqs = np.fft.rfftfreq(nfft, d=1/sr)
        
        f0 = 250.0
        frequencies = np.arange(f0, sr/2, f0)
        amplitudes = np.interp(frequencies,
                               power_spectrum_freqs, 
                               np.sqrt(power_spectrum))
        signal = util_stimuli.complex_tone(f0,
                                           sr,
                                           dur,
                                           harmonic_numbers=None,
                                           frequencies=frequencies,
                                           amplitudes=amplitudes,
                                           phase_mode='sine',
                                           offset_start=True,
                                           strict_nyquist=True)
#         signal = util_stimuli.impose_power_spectrum(signal, power_spectrum)
        if 'noise' in key:
            signal = np.random.randn(nfft)
            signal = util_stimuli.impose_power_spectrum(signal, power_spectrum)
        
        kwargs_plot = {
            'ls': '-',
            'color': 'b',
        }
        if 'noise' in key:
            kwargs_plot['ls'] = '-'
            kwargs_plot['color'] = 'k'
        
        fxx, pxx = util_stimuli.power_spectrum(signal, sr)
        ax.plot(fxx, pxx-pxx.max(), lw=0.25, color='m')
        
        power_spectrum = 10*np.log10(power_spectrum)
        ax.plot(fxx, power_spectrum-power_spectrum.max(), lw=0.5, **kwargs_plot)

        MEAN_FXX = np.array(results_dict[key]['mean_power_spectrum_freqs'])
        MEAN_PXX = np.array(results_dict[key]['mean_power_spectrum'])
#         MEAN_FXX = np.fft.rfftfreq(nfft, d=1/sr)
#         MEAN_PXX = util_stimuli.get_power_spectrum_from_mfcc(mfcc_mean, Minv)
#         MEAN_PXX = 10*np.log10(MEAN_PXX)
        
        ax.plot(MEAN_FXX, MEAN_PXX-MEAN_PXX.max(), lw=2.5, **kwargs_plot)
        
#         mfcc2 = util_stimuli.get_mfcc(signal, M)
#         mfcc2[12:] = 0
#         pxx2 = util_stimuli.get_power_spectrum_from_mfcc(mfcc2, Minv)
#         pxx2 = 10*np.log10(pxx2)
#         fxx2 = np.fft.rfftfreq(len(signal), d=1/sr)
#         ax.plot(fxx2, pxx2-pxx2.max(), color='g')

# Use this function to specify axis scaling, limits, labels, etc.
ax = util_figures.format_axes(ax,
                              xscale='linear',
                              str_xlabel='Frequency (Hz)',
                              str_ylabel='Power (dB)',
                              xlimits=[40, None],
                              ylimits=[-80, None],
                              spines_to_hide=['right', 'top'])
plt.show()

ipd.display(ipd.Audio(signal, rate=sr))


In [ ]:
import stimuli_generate_random_synthetic_tones
import importlib
importlib.reload(stimuli_generate_random_synthetic_tones)


spectral_statistics_filename = '/om/scratch/Mon/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/SPECTRAL_STATISTICS_v00/results_dict.json'

stimuli_generate_random_synthetic_tones.spectrally_shaped_synthetic_dataset(
    'tmp.hdf5',
    500,
    spectral_statistics_filename,
    fs=32e3,
    dur=0.150,
    phase_modes=['cos'],
    range_f0=[80.0, 1001.3713909809752],
    range_snr=[-10., 10.],
    range_dbspl=[30., 90.],
    n_mfcc=12,
    invert_signal_filter=0,
    invert_noise_filter=False,
    generate_signal_in_fft_domain=False,
    out_combined_key='stimuli/signal_in_noise',
    out_signal_key='stimuli/signal',
    out_noise_key='stimuli/noise',
    out_snr_key='snr',
    out_augmentation_prefix='augmentation/',
    random_seed=0,
    disp_step=50)


In [ ]:
import sys
import os
import h5py
import json
import glob
import copy
import pdb
import librosa
import numpy as np
import scipy.signal
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/packages/msutil')
import util_stimuli
import util_misc
import util_figures

key_list = ['stimuli/signal_in_noise']#, 'stimuli/signal', 'stimuli/noise']
with h5py.File('tmp.hdf5', 'r') as f:
    sr = f['sr'][0]
    y = {}
    for k in key_list:
        y[k] = f[k][np.random.randint(500)]
    for k in util_misc.get_hdf5_dataset_key_list(f):
        print(k, f[k])

ipd.display(ipd.Audio(y['stimuli/signal_in_noise'], rate=sr))

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(7.5, 5.0))
for k in key_list:
    fxx, pxx = util_stimuli.power_spectrum(y[k], sr)
    ax.plot(fxx, pxx-pxx.max(), lw=0.5, label=k)

ax.legend()
ax = util_figures.format_axes(ax,
                              xscale='linear',
                              str_xlabel='Frequency (Hz)',
                              str_ylabel='Power (dB SPL)',
                              xlimits=[40, None],
                              ylimits=[-60, None],
                              spines_to_hide=['right', 'top'])
plt.show()


In [ ]:
import sys
import os
import h5py
import glob
import numpy as np
import scipy.signal
import scipy.fftpack
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/packages/msutil')
import util_stimuli
import util_misc
import util_figures


regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/PND_sr32000_v08_*.hdf5'
regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_mfcc/PNDv08PYSmatched12_TLASmatched12_snr_neg10pos10_phase3/stim_0000000-0002100.hdf5'

list_fn = sorted(glob.glob(regex_fn))
fn = list_fn[-1]

key_y = 'stimuli/signal'
key_f0 = 'nopad_f0_mean'
key_f0 = 'f0'

with h5py.File(fn, 'r') as f:
    sr = f['sr'][0]
    IDX = np.random.randint(0, f[key_f0].shape[0])
    f0 = f[key_f0][IDX]
    y = util_stimuli.set_dBSPL(f[key_y][IDX], 60.0)


fxx, pxx = util_stimuli.power_spectrum(y, sr)

print(f0)
harmonic_frequencies = np.arange(f0, sr/2, f0)
IDX = np.digitize(harmonic_frequencies, fxx)
harmonic_freq_bins = fxx[IDX]
spectrum_freq_bins = pxx[IDX]
envelope_spectrum = np.interp(fxx, harmonic_freq_bins, spectrum_freq_bins)

# philbert = np.abs(scipy.signal.hilbert(pxx+50))

fenv, penv = util_stimuli.get_spectral_envelope_lp(y, sr, M=12)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
ax.plot(fxx, pxx, color='k', lw=1.0)
ax.plot(fxx, envelope_spectrum, color='r', lw=2.0)
# ax.plot(fenv, penv, color='r', lw=1.0)
ax = util_figures.format_axes(ax,
                              xscale='linear',
                              str_xlabel='Frequency (Hz)',
                              str_ylabel='Power (dB SPL)',
                              xlimits=[40, sr/2],
                              ylimits=[-20, None],
                              spines_to_hide=['right', 'top'])
plt.show()

ipd.display(ipd.Audio(y, rate=sr))


In [ ]:
# import sys
# import os
# import h5py
# import json
# import glob
# import numpy as np
# import scipy.signal
# %matplotlib inline
# import matplotlib.pyplot as plt
# import IPython.display as ipd
# sys.path.append('/packages/msutil')
# import util_stimuli
# import util_misc
# import util_figures

import importlib
import stimuli_compute_statistics
importlib.reload(stimuli_compute_statistics)

import stimuli_analyze_pystraight
importlib.reload(stimuli_analyze_pystraight)

# regex_fn = '/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/PYSTRAIGHT_v01_foreground/PND*.hdf5'
regex_fn = '/om/scratch/Fri/msaddler/data_pitchnet/PND_mfcc/PNDv08PYSnegated12_TLASmatched12_snr_neg10pos10_phase3/PYSTRAIGHT_v01_foreground/*.hdf5'
print(regex_fn)
stimuli_analyze_pystraight.summarize_pystraight_statistics(
    regex_fn,
    fn_results='results_dict.json',
    key_sr='sr',
    key_signal_list=['stimuli/signal'])

# regex_fn = '/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/SPECTRAL_STATISTICS_v00/PND*.hdf5'
# regex_fn = '/om/scratch/Fri/msaddler/data_pitchnet/PND_mfcc/PNDv08PYSmatched12_TLASmatched12_snr_neg10pos10_phase3/SPECTRAL_STATISTICS_v00/*.hdf5'
# print(regex_fn)
# stimuli_compute_statistics.summarize_spectral_statistics(regex_fn,
#                                   fn_results='results_dict.json',
#                                   key_sr='sr',
#                                   key_f0=None,
#                                   key_signal_list=['stimuli/signal', 'stimuli/noise'])


In [ ]:
import sys
import os
import h5py
import glob
import numpy as np
import scipy.signal
import scipy.fftpack
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/packages/msutil')
import util_stimuli
import util_misc
import util_figures


regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/PYSTRAIGHT_v01_foreground/*.hdf5'
# regex_fn = '/om/scratch/*/msaddler/data_pitchnet/PND_mfcc/debug_PNDv08PYSmatched12_TLASmatched12_snr_neg10pos10_phase3/PYSTRAIGHT_v01_foreground/*.hdf5'
list_fn = sorted(glob.glob(regex_fn))
fn = list_fn[0]


key_f0 = 'f0'
key_y = 'stimuli/signal_INTERP_interp_signal'

with h5py.File(fn, 'r') as f:
    for k in util_misc.get_hdf5_dataset_key_list(f):
        print(k, f[k].shape)
    sr = f['sr'][0]
    IDX = np.random.randint(0, f[key_y].shape[0])
#     IDX = 5
    y = util_stimuli.set_dBSPL(f[key_y][IDX], 60.0)
    if True:#key_f0 in f:
        f0 = f[key_f0][IDX]
        print('------------>', f0, f['pystraight_success'][IDX])
    
    NTMP = f[key_y].shape[1]
    power = 0
    while NTMP > 2:
        NTMP /= 2
        power += 1
    n_fft = int(2 ** power)
    M = librosa.filters.mel(sr, n_fft, n_mels=40)
    Minv = np.linalg.pinv(M)
    
    fxx = np.fft.rfftfreq(n_fft, d=1/sr)
    pxx = f['stimuli/signal_FILTER_spectrumSTRAIGHT'][IDX]
    mfcc = f['stimuli/signal_FILTER_spectrumSTRAIGHT_mfcc'][IDX]
#     mfcc = scipy.fftpack.dct(np.log(np.matmul(M, pxx)), norm='ortho')
    mfcc[12:] = 0
    pxx_mfcc = np.matmul(Minv, np.exp(scipy.fftpack.idct(mfcc, norm='ortho')))
    pxx_mfcc[pxx_mfcc < 0] = 0
    pxx_mfcc = 10*np.log10(pxx_mfcc)
    pxx_mfcc -= pxx_mfcc.max()
    
    pxx_straight = pxx
    pxx_straight = 10*np.log10(pxx_straight)
    pxx_straight -= pxx_straight.max()

fyy, pyy = util_stimuli.power_spectrum(y, sr)
pyy -= pyy.max()


fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 5))
ax.plot(fyy, pyy, color='k', lw=1.0)
ax.plot(fxx, pxx_straight, color='r', lw=2.0)
ax.plot(fxx, pxx_mfcc, color='g', lw=2.0)

ax = util_figures.format_axes(ax,
                              xscale='linear',
                              str_xlabel='Frequency (Hz)',
                              str_ylabel='Power (dB SPL)',
                              xlimits=[40, sr/2],
#                               ylimits=[-100, 10],
                              spines_to_hide=['right', 'top'])
plt.show()

ipd.display(ipd.Audio(y, rate=sr))


In [ ]:
import sys
import os
import h5py
import json
import glob
import numpy as np
import librosa
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

sys.path.append('/packages/msutil')
import util_stimuli
import util_misc
import util_figures

data_dir = '/om/scratch/Fri/msaddler/data_pitchnet/'
list_dataset_tag = [
#     ('PND_v08/noise_TLAS_snr_neg10pos10/SPECTRAL_STATISTICS_v00/results_dict.json', 'Natural power spectrum'),
    ('PND_mfcc/debug_PNDv08PYSmatched12_TLASmatched12_snr_neg10pos10_phase3/SPECTRAL_STATISTICS_v00/results_dict.json', 'Synthetic power spectrum (12-MFCC-matched to natural)'),

#     ('PND_v08/noise_TLAS_snr_neg10pos10/PYSTRAIGHT_v01_foreground/results_dict.json', 'Natural filter spectrum'),
#     ('PND_mfcc/debug_PNDv08PYSmatched12_TLASmatched12_snr_neg10pos10_phase3/SPECTRAL_STATISTICS_v00/results_dict.json', 'Synthetic filter spectrum (12-MFCC-matched to natural)'),
#     ('PND_mfcc/debug_PNDv08PYSnegated12_TLASmatched12_snr_neg10pos10_phase3/SPECTRAL_STATISTICS_v00/results_dict.json', 'Synthetic filter spectrum (12-MFCC-matched to natural)'),

#     ('PND_v08/noise_TLAS_snr_neg10pos10_filter_signalLPv01/PYSTRAIGHT_v01_foreground/results_dict.json', 'Natural foreground (lowpass-filtered)'),
#     ('PND_v08/noise_TLAS_snr_neg10pos10_filter_signalHPv00/PYSTRAIGHT_v01_foreground/results_dict.json', 'Natural foreground (highpass-filtered)'),

#     ('PND_v08/noise_TLAS_snr_neg10pos10/SPECTRAL_STATISTICS_v00/results_dict.json', 'Natural'),
#     ('PND_v08/noise_TLAS_snr_neg10pos10_filter_signalLPv00/SPECTRAL_STATISTICS_v00/results_dict.json', 'Natural (lowpass_v00)'),
#     ('PND_v08/noise_TLAS_snr_neg10pos10_filter_signalLPv01/SPECTRAL_STATISTICS_v00/results_dict.json', 'Natural (lowpass_v01)'),
#     ('PND_v08/noise_TLAS_snr_neg10pos10_filter_signalHPv00/SPECTRAL_STATISTICS_v00/results_dict.json', 'Natural (highpass_v00)'),
#     ('PND_v08/noise_TLAS_snr_neg10pos10/PYSTRAIGHT_v01_foreground/results_dict.json', 'Natural'),
#     ('PND_v08/noise_TLAS_snr_neg10pos10_filter_signalLPv00/PYSTRAIGHT_v01_foreground/results_dict.json', 'Natural (lowpass_v00)'),
#     ('PND_v08/noise_TLAS_snr_neg10pos10_filter_signalLPv01/PYSTRAIGHT_v01_foreground/results_dict.json', 'Natural (lowpass_v01)'),
#     ('PND_v08/noise_TLAS_snr_neg10pos10_filter_signalHPv00/PYSTRAIGHT_v01_foreground/results_dict.json', 'Natural (highpass_v00)'),

#     ('PND_mfcc/PNDv08matched12_TLASmatched12_snr_neg10pos10_phase0/SPECTRAL_STATISTICS_v00/results_dict.json', 'Synthetic foreground (12-MFCC-matched to natural)'),
#     ('PND_mfcc/debugPNDv08negated12_TLASmatched12_snr_neg10pos10_phase0/SPECTRAL_STATISTICS_v00/results_dict.json', 'Synthetic foreground (12-MFCC-negated to natural)'),
#     ('PND_v08spch/noise_TLAS_snr_neg10pos10/PYSTRAIGHT_v01_foreground/results_dict.json', 'Natural speech'),
#     ('PND_v08inst/noise_TLAS_snr_neg10pos10/PYSTRAIGHT_v01_foreground/results_dict.json', 'Natural instruments'),
]

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(8, 4))

clist = 'krbgmyc'
for cidx, (dataset_tag, label_tag) in enumerate(list_dataset_tag):
    fn_results_dict = os.path.join(data_dir, dataset_tag)
    print(fn_results_dict)
    with open(fn_results_dict, 'r') as f:
        results_dict = json.load(f)
    
    if 'PYSTRAIGHT_v01_foreground' in fn_results_dict:
        key_fxx = 'mean_filter_spectrum_freqs'
        key_pxx = 'mean_filter_spectrum'
        key_n_fft = 'mean_filter_spectrum_n_fft'
    else:
        key_fxx = 'mean_power_spectrum_freqs'
        key_pxx = 'mean_power_spectrum'
        key_n_fft = 'mean_power_spectrum_n_fft'
    
    for key in sorted(results_dict.keys()):
        MEAN_FXX = np.array(results_dict[key][key_fxx])
        MEAN_PXX = np.array(results_dict[key][key_pxx])
        if 'PYSTRAIGHT_v01_foreground' in fn_results_dict:
            MEAN_PXX -= 10*np.log10(20e-6)
        
        sr = results_dict[key]['sr']
        mfcc_mean = np.array(results_dict[key]['mfcc_mean'])
        mfcc_mean[12:0]
        mfcc_cov = np.array(results_dict[key]['mfcc_cov'])
        n_fft = np.array(results_dict[key][key_n_fft])
        M = librosa.filters.mel(sr, n_fft, n_mels=mfcc_mean.shape[0])
        Minv = np.linalg.pinv(M)
        
        kwargs_plot = {
            'ls': '-',
            'color': clist[cidx],
            'label': '{} : {}'.format(label_tag, key),
        }
        if 'noise' in key:
            kwargs_plot['ls'] = '--'
            kwargs_plot['color'] = [0.5] * 3
            kwargs_plot['label'] = None
        
        ax.plot(MEAN_FXX,
                MEAN_PXX,#-MEAN_PXX.max(),
                **kwargs_plot)
        
#         PXX_MFCC = 10*np.log10(util_stimuli.get_power_spectrum_from_mfcc(mfcc_mean, Minv))
#         ax.plot(MEAN_FXX,
#                 PXX_MFCC,#-PXX_MFCC.max(),
#                 **kwargs_plot)
        
#         sample_PXX_MFCC = np.zeros_like(PXX_MFCC)
#         nsamples=50
#         for _ in range(nsamples):
#             mfcc = np.random.multivariate_normal(mfcc_mean, mfcc_cov)
#             mfcc[6:] = 0
#             sample_PXX_MFCC += 10*np.log10(util_stimuli.get_power_spectrum_from_mfcc(mfcc, Minv))
#         sample_PXX_MFCC /= nsamples
#         ax.plot(MEAN_FXX,
#                 sample_PXX_MFCC-sample_PXX_MFCC.max(),
#                 **kwargs_plot)
    
ax.legend(loc='upper right', ncol=1)
ax = util_figures.format_axes(ax,
                              xscale='log',
                              str_xlabel='Frequency (Hz)',
                              str_ylabel='Power (dB)',
                              xlimits=[40, None],
                              ylimits=[-20, None],
                              spines_to_hide=['right', 'top'])
plt.show()


In [ ]:
import sys
import h5py
import numpy as np
import IPython.display as ipd
sys.path.append('/packages/msutil')
import util_misc
import util_stimuli


# fn = '/om/user/msaddler/data_pitchnet/neurophysiology/bernox2005_SlidingFixedFilter_lharm01to30_phase0_f0min080_f0max320/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/bez2018meanrates_009216-012288.hdf5'
# fn = '/om/scratch/Fri/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/sr20000_cf100_species002_spont070_BW10eN1_IHC3000Hz_IHC7order/bez2018meanrates_098_007000-014000.hdf5'
# fn = '/om/scratch/Fri/msaddler/data_pitchnet/PND_v08inst/noise_TLAS_snr_neg10pos10/PND_sr32000_v08inst_1422630-1437000.hdf5'
fn = '/om/scratch/Fri/msaddler/data_pitchnet/PND_v08spch/noise_TLAS_snr_neg10pos10/PND_sr32000_v08spch_1422630-1437000.hdf5'
# fn_new = 'PND_v08inst_examples_for_metamers.hdf5'
fn_new = 'PND_v08spch_examples_for_metamers.hdf5'

np.random.seed(998)

data_dict = {}
with h5py.File(fn, 'r') as f:
    for k in util_misc.get_hdf5_dataset_key_list(f):
#         print(k, f[k])
        if f[k].shape[0] == 1:
            data_dict[k] = f[k][:]

    sr = f['sr'][0]
    
    key_signal = 'stimuli/signal'
    N = 15
    for itrN in range(N):
        IDX = np.random.randint(low=0, high=f['nopad_f0_mean'].shape[0])
        for k in util_misc.get_hdf5_dataset_key_list(f):
            if f[k].shape[0] > 1:
                if k not in data_dict:
                    data_dict[k] = []
                data_dict[k].append(f[k][IDX])
        
        idx_start = f['nopad_start_index'][IDX] - f['segment_start_index'][IDX]
        idx_end =  f['nopad_end_index'][IDX] - f['segment_start_index'][IDX]
        y = f['stimuli/signal'][IDX, idx_start:idx_end]
        y_preprocessed = y[0:int(0.05*sr)]
        y_preprocessed = util_stimuli.set_dBSPL(y_preprocessed, 60.0)
        
        if itrN == 0:
            data_dict['y'] = []
            data_dict['y_preprocessed'] = []
            data_dict['f0'] = []
        data_dict['y'].append(y)
        data_dict['y_preprocessed'].append(y_preprocessed)
        data_dict['f0'].append(f['nopad_f0_mean'][IDX])
        
#         ipd.display(ipd.Audio(y_preprocessed, rate=sr))

# f_new = h5py.File(fn_new, 'w')
# for k in sorted(data_dict.keys()):
#     data_dict[k] = np.array(data_dict[k])
# #     print(k, data_dict[k].shape, data_dict[k].dtype)
#     f_new.create_dataset(k, data=data_dict[k])
# f_new.close()

print('F0:', data_dict['f0'])


In [ ]:
import sys
import h5py
import numpy as np
import IPython.display as ipd
sys.path.append('/packages/msutil')
import util_misc
import util_stimuli

fn = '/om4/group/mcdermott/user/msaddler/pitchnet_dataset/pitchnetDataset/assets/data/interim/swcDataframe_interim_processed2019-07-15-1830_processedFile__noNaN_sr32000.pdh5'
fn = '/om4/group/mcdermott/user/msaddler/pitchnet_dataset/pitchnetDataset/assets/data/interim/sr32000_pystraight/swc_183928-184492.hdf5'
with h5py.File(fn) as f:
    for k in util_misc.get_hdf5_dataset_key_list(f):
        print(k, f[k])
    
    for _ in range(10):
        IDX = np.random.randint(f['interp_signal'].shape[0])
        y = f['interp_signal'][IDX]
        sr = f['sr'][0]

        ipd.display(ipd.Audio(y, rate=sr))


In [ ]:
import sys
import h5py
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd
sys.path.append('/packages/msutil')
import util_misc
import util_stimuli

# fn = '/om/user/msaddler/data_pitchnet/bernox2005/lowharm_v01/stim.hdf5'
fn = '/om/user/msaddler/data_pitchnet/bernox2005/neurophysiology_v01_EqualAmpTEN_lharm01to15_phase0_f0min080_f0max640/stim.hdf5'
fn = '/om/user/msaddler/data_pitchnet/bernox2005/neurophysiology_v01_EqualAmpTEN_lharm01to30_phase0_f0min080_f0max320/stim.hdf5'
with h5py.File(fn, 'r') as f:
    for k in util_misc.get_hdf5_dataset_key_list(f):
        print(k, f[k])
    
    print(np.unique(f['max_audible_harm'][:]))
    base_f0 = f['base_f0'][:]
    print(np.unique(base_f0).shape, base_f0.min(), base_f0.max())
    
    IDX = -10000
    sr = f['config_tone/fs'][0]
    y = f['tone_in_noise'][IDX]
    
    print(util_stimuli.get_dBSPL(y))
    fxx, pxx = util_stimuli.power_spectrum(y, sr)
    
    fig, ax = plt.subplots(figsize=(12, 2))
    ax.plot(fxx, pxx)
    ax.set_xlim([0, sr/2])
    ax.set_ylim([-30, None])
    plt.show()
    
    ipd.display(ipd.Audio(y, rate=sr))


In [1]:
import importlib
import stimuli_generate_BernsteinOxenhamPureTone
importlib.reload(stimuli_generate_BernsteinOxenhamPureTone)

# hdf5_filename = '/om/user/msaddler/data_pitchnet/bernox2005/puretone_v01/stim.hdf5'
# stimuli_generate_BernsteinOxenhamPureTone.main(
#     hdf5_filename,
#      fs=32e3,
#      dur=0.150,
#      f0_min=80.0,
#      f0_max=10240.0,
#      f0_n=50,
#      dbspl_min=20.0,
#      dbspl_max=60.0,
#      dbspl_step=0.25,
#      noise_dBHzSPL=10.0,
#      noise_attenuation_start=600.0,
#      noise_attenuation_slope=2,
#      disp_step=100)

# hdf5_filename = '/om/user/msaddler/data_pitchnet/bernox2005/puretone_v02/stim.hdf5'
# stimuli_generate_BernsteinOxenhamPureTone.main(
#     hdf5_filename,
#      fs=32e3,
#      dur=0.150,
#      f0_min=80.0,
#      f0_max=10240.0,
#      f0_n=50,
#      dbspl_min=20.0,
#      dbspl_max=60.0,
#      dbspl_step=0.25,
#      noise_dBHzSPL=12.0,
#      noise_attenuation_start=600.0,
#      noise_attenuation_slope=2,
#      disp_step=100)

# hdf5_filename = '/om/user/msaddler/data_pitchnet/bernox2005/puretone_v03/stim.hdf5'
# stimuli_generate_BernsteinOxenhamPureTone.main(
#     hdf5_filename,
#      fs=32e3,
#      dur=0.150,
#      f0_min=80.0,
#      f0_max=10240.0,
#      f0_n=50,
#      dbspl_min=20.0,
#      dbspl_max=60.0,
#      dbspl_step=0.25,
#      noise_dBHzSPL=8.0,
#      noise_attenuation_start=600.0,
#      noise_attenuation_slope=2,
#      disp_step=100)


ImportError in `dataset_util.py` No module named 'pyfftw'
[INITIALIZING]: /om/user/msaddler/data_pitchnet/bernox2005/puretone_v01/stim.hdf5
... signal 000000 of 008050, f0=80.00, dbspl=20.00
... signal 000100 of 008050, f0=80.00, dbspl=45.00
... signal 000200 of 008050, f0=88.33, dbspl=29.75
... signal 000300 of 008050, f0=88.33, dbspl=54.75
... signal 000400 of 008050, f0=97.52, dbspl=39.50
... signal 000500 of 008050, f0=107.67, dbspl=24.25
... signal 000600 of 008050, f0=107.67, dbspl=49.25
... signal 000700 of 008050, f0=118.88, dbspl=34.00
... signal 000800 of 008050, f0=118.88, dbspl=59.00
... signal 000900 of 008050, f0=131.25, dbspl=43.75
... signal 001000 of 008050, f0=144.92, dbspl=28.50
... signal 001100 of 008050, f0=144.92, dbspl=53.50
... signal 001200 of 008050, f0=160.00, dbspl=38.25
... signal 001300 of 008050, f0=176.65, dbspl=23.00
... signal 001400 of 008050, f0=176.65, dbspl=48.00
... signal 001500 of 008050, f0=195.04, dbspl=32.75
... signal 001600 of 008050, f0=1